### At the begginig a bit preprocessing of data: 

In [1]:
import numpy as np
import pandas as pd
import Bio as bio
from os import listdir
from os.path import isfile, join

In [2]:
csv_files = [f for f in listdir('csv_data_math') if isfile(join('csv_data_math', f))]
for i in range(len(csv_files)):
    csv_files[i] = 'csv_data_math/' + csv_files[i]
    
csv_files[0:5]

['csv_data_math/CLMD_trachomatis.csv',
 'csv_data_math/CLOS_difficile.csv',
 'csv_data_math/CMV_AD169.csv',
 'csv_data_math/CPBT_jejuni.csv',
 'csv_data_math/EBV_AG876.csv']

In [3]:
Proteins_data = pd.read_csv(csv_files.pop(0),  index_col=0)
#Proteins_data.head(20)

In [4]:
for x in csv_files:
    res_data = pd.read_csv(x,  index_col=0)
    Proteins_data = pd.concat([Proteins_data, res_data], ignore_index=True)

In [5]:
Proteins_data.shape

(106549, 402)

In [6]:
# Be careful with your data

Data_anal = Proteins_data

labels_categorized = Data_anal[Data_anal.columns[0]].values
feature_matrix = Data_anal[Data_anal.columns[2:22]].values

feature_matrix[0]

# Oh gosh

array([-0.67834601,  0.70445678, -0.71153675,  4.95243735,  0.70445678,
       -0.71153675,  2.1204503 , -0.71153675,  0.70445678,  0.70445678,
        0.70445678, -0.71153675, -0.71153675, -0.71153675, -0.71153675,
       -0.71153675, -0.71153675, -0.71153675,  0.70445678, -0.71153675])

In [7]:
#labels_categorized

In [8]:
# Making labels not categorized (slightly)

labels = []

for x in labels_categorized:
    if x == 'human_proteome':
        labels.append(1)
    else:
        labels.append(0)
        
Data_anal['Label'] = labels

In [9]:
labels.count(1)

74349

### Looking at our data:

In [10]:
### Here is the place for your code!!!

### Starting to make base-line with easy methods:

In [20]:
# Importing SKlearn magic

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [12]:
# Making 2 Train-Test split
# One (final_test_*) for the final test and second (test_*) for train-test

train_feature_matrix, test_feature_matrix,\
train_labels, test_labels = train_test_split(feature_matrix, labels, test_size=0.2, random_state=42)

In [13]:
# Data preprocessing
# We will just give it try
#scaler = preprocessing.StandardScaler()
#train_feature_matrix_scaled = scaler.fit_transform(train_feature_matrix)
#test_feature_matrix_scaled = scaler.fit_transform(test_feature_matrix)
print('t h e r e  i s  n o t h i n g  i n t e r e s t i n g')

t h e r e  i s  n o t h i n g  i n t e r e s t i n g


#### Method 1: Random Forest

In [17]:
# Will try Random Forest

forest = RandomForestClassifier(n_estimators=5000, max_depth=15, \
                                max_features='sqrt', bootstrap='True', n_jobs=-1)

In [18]:
# Training 
forest.fit(train_feature_matrix, train_labels)

RandomForestClassifier(bootstrap='True', class_weight=None, criterion='gini',
            max_depth=15, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
# Score for final_test

forest.score(test_feature_matrix, test_labels)

0.8203190990145471

In [21]:
y_pred = forest.predict(test_feature_matrix)

In [22]:
print(classification_report(test_labels, y_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.82      0.50      0.63      6330
           1       0.82      0.95      0.88     14980

   micro avg       0.82      0.82      0.82     21310
   macro avg       0.82      0.73      0.75     21310
weighted avg       0.82      0.82      0.81     21310



#### Method 2: Logistic Regression

In [ ]:
# Making LogRes class

lr_clf = LogisticRegression(max_iter=30000, solver='lbfgs', tol=0.00001, penalty ='l2',
                            C=1, class_weight = { 1 : 1 , 0: 1})

In [ ]:
# Fitting not preproccesed data

lr_clf.fit(train_feature_matrix, train_labels)

In [ ]:
# Weights

lr_clf.coef_

In [ ]:
# Score for test (Accuracy and F1-score)

accuracy_log = lr_clf.score(test_feature_matrix, test_labels)

prediction = lr_clf.predict(test_feature_matrix)

f_1 = f1_score(prediction, test_labels)

print('Acuracy = '+str(accuracy_log))
print('F1 score  = '+str(f_1))

#### Method 3: SVM

In [ ]:
# Making

svm_rbf = SVC(kernel='rbf', max_iter=20000, tol=0.0001, C=1.5, gamma=2)

parametrs_rbf = {'C':np.linspace(1, 20, 5), 'gamma':np.linspace(0, 10, 5)}

#grid_search_rbf = GridSearchCV(svm_rbf, parametrs_rbf, cv = 5, scoring = 'accuracy', n_jobs=-1)
#grid_search_rbf.fit(train_feature_matrix, train_labels)

In [ ]:
# Fitting not preproccesed data

svm_rbf.fit(train_feature_matrix, train_labels)

In [ ]:
# Score for test (Accuracy and F1-score)

svm_rbf.score(train_feature_matrix, train_labels)

In [ ]:
svm_rbf.score(test_feature_matrix, test_labels)

#### Method 4: Gradient boosting

In [23]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [24]:
# Making

Gradboost = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=10),
    n_estimators=1000,
    random_state=42)

In [25]:
# Fitting not preproccesed data

Gradboost.fit(train_feature_matrix, train_labels)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=1000, random_state=42)

In [26]:
# Score for test (Accuracy and F1-score)

accuracy_log = Gradboost.score(test_feature_matrix, test_labels)

prediction = Gradboost.predict(test_feature_matrix)

f_1 = f1_score(prediction, test_labels)

print('Acuracy = '+str(accuracy_log))
print('F1 score  = '+str(f_1))

Acuracy = 0.796245893946504
F1 score  = 0.8589527027027027


In [27]:
print(classification_report(test_labels, prediction, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.68      0.59      0.63      6330
           1       0.84      0.88      0.86     14980

   micro avg       0.80      0.80      0.80     21310
   macro avg       0.76      0.74      0.75     21310
weighted avg       0.79      0.80      0.79     21310

